In [21]:
import pandas as pd
import numpy as np

In [22]:
sch_flow = pd.read_csv("data/raw/schwartau/flow_schwartau.csv")
test_flow = sch_flow
sch_temp = pd.read_csv("data/raw/schwartau/temperature_schwartau.csv")
test_sch = sch_temp
sch_humid = pd.read_csv("data/raw/schwartau/humidity_schwartau.csv")
sch_weight = pd.read_csv("data/raw/schwartau/weight_schwartau.csv")

pls love me I need it to survive -Derek

<hr>
Cleaning notes for flow:

* Lots of zero data
* Measured every minute. Need to aggregate
* Negative numbers are bee departures
* Positive numbers are bee arrivals
* Keep arrivals and departures separate 

I separated by every 15 like the temp data and then created 2 df copies for all arrivals and all departures.  DONE-Derek

In [23]:
#original flow
print("Original dataframe: ")
print(test_flow.head())
print()

#Required changes:
#Rather than dropping all data that doesn't occur every 15 minutes, we need to sum up departures and arrivals
#that occur in each 15 minute timespan. 

#Track which indexes should be dropped
dropList = []

#Drop N/A data points
sch_flow = sch_flow.dropna()

#Convert the timestamps to date time objects
sch_flow['timestamp'] = pd.to_datetime(sch_flow['timestamp'])

#Create columns for departures and arrivals
sch_flow['departures'] = sch_flow['flow']
sch_flow['arrivals'] = sch_flow['flow']

#Counters for arrivals in each 15 minute interval
arrivals = 0
departures = 0
total_flow = 0

#Set the length of the interval
interval_length = 15

for ind,minute in sch_flow['timestamp'].dt.minute.items():   
    #Count the toal flow, departures, and arrival for the current interval
    current_flow = sch_flow.at[ind, 'flow']
    total_flow += current_flow
    if(current_flow < 0):
        #Departures are negative flow numbers
        departures += abs(current_flow)
    else:
        arrivals += current_flow
    
    #Set the values at each 15 minute interval
    if(minute % interval_length == 0):
        #Set total flow
        sch_flow.at[ind, 'flow'] = total_flow
        
        #Set departures 
        sch_flow.at[ind, 'departures'] = departures
        
        #Set arrivals
        sch_flow.at[ind, 'arrivals'] = arrivals
        
        #Reset the counters
        total_flow = 0
        departures = 0
        arrivals = 0
    
    #Mark indexes that aren't at 15 minute intervals to be dropped
    if (minute % interval_length != 0):
        dropList.append(ind)

#Drop indexes marked for dropping
sch_flow = sch_flow.drop(index = dropList)

sch_flow.reset_index(drop=True, inplace=True)

#Display some information about the new dataframe
print("New dataframe:")
print(sch_flow.head())
print()


#Do a quick sanity check that the above operation actually worked
#Testing your code is always good!
flow_problem_flag = False #Detect problems with the flow calculations
filtering_problem_flag = False #Detect problems with filtering out minutes that arent at 15 minute intervals
problem_index = 0 #The index where the first problem occured
flow_gotten = 0 # The number that the flow actually came out to
flow_actual = 0 # The actual flow
for ind,minute in sch_flow['timestamp'].dt.minute.items():
    
    #Add the arrivals and departures
    #These should be the same as the total flow
    current_count = 0
    current_count += sch_flow.at[ind, 'arrivals'] #Arrivals are counted as positive 'flow'
    current_count -= sch_flow.at[ind, 'departures'] #Departures are counted as negative 'flow'
    
    #Check that arrivals are 
    if(current_count != sch_flow.at[ind, 'flow']):
        flow_problem_flag = True
        problem_index = ind
        flow_gotten = current_count
        flow_actual = sch_flow.at[ind, 'flow']
        break
    
    if(minute % interval_length != 0):
        filtering_problem_flag = True
        problem_index = ind
        break
    
if(flow_problem_flag):
    print("Looks like you have a flow problem at index: " + str(problem_index) + "! Have fun with that...")
    print("The flow count gotten: " + str(flow_gotten))
    print("The flow count expected: " + str(flow_actual))
elif(filtering_problem_flag):
    print("Looks like you have a time filtering problem at index: " + str(problem_index) + "! Have fun with that...")
else:
    print("You are a coding master! No problems detected!")

Original dataframe: 
             timestamp  flow
0  2017-01-01 14:15:00     0
1  2017-01-01 14:16:00     0
2  2017-01-01 14:17:00     0
3  2017-01-01 14:18:00     0
4  2017-01-01 14:19:00     0

New dataframe:
            timestamp  flow  departures  arrivals
0 2017-01-01 14:15:00     0           0         0
1 2017-01-01 14:30:00     0           0         0
2 2017-01-01 14:45:00     0           0         0
3 2017-01-01 15:00:00     0           0         0
4 2017-01-01 15:15:00     0           0         0

You are a coding master! No problems detected!


<hr>
Cleaning notes for temperature:

* May contain NaN (Dropped rows containing these.  DONE-Derek)
* Measured in degrees celsius (Will leave as C unless otherwise stated.  DONE-Derek)
* Measured every 5 minutes (coverted to every 15)

In [24]:
#Preview of temp
print(test_sch.head())
print()
#Initial number of values
print(len(test_sch.index))
#Drop NaN value rows
sch_tempFilter = test_sch.dropna()
print()
#Number of values after
print(len(sch_tempFilter.index))
print()
print(sch_tempFilter.head())
print()

#Sorting timestamp...
tempList = []
sch_temp = sch_temp.dropna()
sch_temp['timestamp'] = pd.to_datetime(sch_temp['timestamp'])
for ind,minute in sch_temp['timestamp'].dt.minute.items():
    if (minute %15 != 0):
        tempList.append(ind)
sch_temp = sch_temp.drop(index = tempList)

#sch_temp.reset_index()

print(sch_temp.head())

             timestamp  temperature
0  2017-01-01 14:10:00          NaN
1  2017-01-01 14:15:00       12.340
2  2017-01-01 14:20:00       12.270
3  2017-01-01 14:25:00       12.276
4  2017-01-01 14:30:00       12.356

253430

251398

             timestamp  temperature
1  2017-01-01 14:15:00       12.340
2  2017-01-01 14:20:00       12.270
3  2017-01-01 14:25:00       12.276
4  2017-01-01 14:30:00       12.356
5  2017-01-01 14:35:00       12.404

             timestamp  temperature
1  2017-01-01 14:15:00       12.340
4  2017-01-01 14:30:00       12.356
7  2017-01-01 14:45:00       12.492
10 2017-01-01 15:00:00       12.454
13 2017-01-01 15:15:00       12.378


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<hr>
Cleaning notes for humidity:

* Measured in percentage
* Measured twice per day
* May need to be suplimented with other data

In [25]:
sch_humid.head()

,timestamp,humidity
0,2017-01-01 13:00:00,98.040310
1,2017-01-02 01:00:00,98.610556
2,2017-01-02 13:00:00,99.002083
3,2017-01-03 01:00:00,98.486806
4,2017-01-03 13:00:00,98.320139


<hr>
Cleaning notes for weight:

* Measured twice per day. Need some creative solution here
* Potential to correlate with flow?
* The weight for schwartau needs to be divided by 1000. It was measured in grams accidentally. DONE-Derek

In [26]:
#original data
test_weight = sch_weight
print(test_weight.head())
print()
#fixing weight
sch_weight["weight"] = sch_weight["weight"].div(1000)
print(sch_weight.head())

             timestamp        weight
0  2017-01-01 13:00:00  50736.790078
1  2017-01-02 01:00:00  50700.685000
2  2017-01-02 13:00:00  50614.907500
3  2017-01-03 01:00:00  50739.824167
4  2017-01-03 13:00:00  50799.746944

             timestamp     weight
0  2017-01-01 13:00:00  50.736790
1  2017-01-02 01:00:00  50.700685
2  2017-01-02 13:00:00  50.614908
3  2017-01-03 01:00:00  50.739824
4  2017-01-03 13:00:00  50.799747


<hr>

Finally, combine the cleaned dataset into one dataframe:

In [27]:
#Display information about the flow dataset
print("\n\nDeparture dataframe:")
print("Departure dataframe length: " + str(len(sch_flow)))
sch_flow.set_index('timestamp', inplace=True)
print(sch_flow.head())

#Display information about the temperature dataset
print("\n\nTemperature dataframe:")
print("Temperature dataframe length: " + str(len(sch_temp)))
sch_temp.set_index('timestamp', inplace=True)
print(sch_temp.head())


sch_merged = sch_flow.join(sch_temp)

#Add timestamp back as a column
#sch_merged['timestamp'] = sch_merged.index

#Display information about the merged dataset
print("\n\nMerged dataset:")
print("Merged dataset length: " + str(len(sch_merged)))
print(sch_merged.head(2000))



Departure dataframe:
Departure dataframe length: 167584
                     flow  departures  arrivals
timestamp                                      
2017-01-01 14:15:00     0           0         0
2017-01-01 14:30:00     0           0         0
2017-01-01 14:45:00     0           0         0
2017-01-01 15:00:00     0           0         0
2017-01-01 15:15:00     0           0         0


Temperature dataframe:
Temperature dataframe length: 83800
                     temperature
timestamp                       
2017-01-01 14:15:00       12.340
2017-01-01 14:30:00       12.356
2017-01-01 14:45:00       12.492
2017-01-01 15:00:00       12.454
2017-01-01 15:15:00       12.378


Merged dataset:
Merged dataset length: 167616
                     flow  departures  arrivals  temperature
timestamp                                                   
2017-01-01 14:15:00     0           0         0       12.340
2017-01-01 14:15:00     0           0         0       12.340
2017-01-01 14:30:00   

In [31]:
sch_merged_final = sch_merged.groupby([sch_merged.index]).agg({'flow': np.sum, 'arrivals': np.sum, 'departures':np.sum, 'temperature': np.mean})
sch_merged_final['timestamp'] = sch_merged_final.index

print(sch_merged_final.head(200))

                     flow  arrivals  departures  temperature  \
timestamp                                                      
2017-01-01 14:15:00     0         0           0       12.340   
2017-01-01 14:30:00     0         0           0       12.356   
2017-01-01 14:45:00     0         0           0       12.492   
2017-01-01 15:00:00     0         0           0       12.454   
2017-01-01 15:15:00     0         0           0       12.378   
...                   ...       ...         ...          ...   
2017-01-03 15:00:00    -1         0           1       14.220   
2017-01-03 15:15:00    -1         0           1       19.180   
2017-01-03 15:30:00    -1         0           1       18.756   
2017-01-03 15:45:00     0         0           0       17.122   
2017-01-03 16:00:00    -1         0           1       21.110   

                              timestamp  
timestamp                                
2017-01-01 14:15:00 2017-01-01 14:15:00  
2017-01-01 14:30:00 2017-01-01 14:30:00  

In [33]:
#Save the data as a CSV file
sch_merged_final.to_csv('data/cleaned/schwartau/schwartau.csv', header=True, index=False)